In [1]:
from dotenv import load_dotenv
import pandas as pd
import os
import ipaddress
import json
import requests
import urllib.parse
load_dotenv()


# Function to check if IP is within the same /24 subnet
def is_same_subnet(ip1, ip2):
    ip1_subnet = ipaddress.ip_network(ip1 + '/24', strict=False)
    return ipaddress.ip_address(ip2) in ip1_subnet

# Function to read proxy list from a file
def read_proxy_list(filename):
    with open(filename, 'r') as file:
        return [line.strip() for line in file.readlines()]

# Function to filter proxy IPs that are not in the same subnet as any bad IPs
def filter_proxies(proxy_list, bad_ip_list):
    filtered_proxies = []
    for proxy in proxy_list:
        proxy_ip = proxy.split('//')[1].split(':')[0]  # Extract IP address from URL
        if not any(is_same_subnet(bad_ip, proxy_ip) for bad_ip in bad_ip_list):
            filtered_proxies.append(proxy)
    return filtered_proxies

In [6]:
max_page = 6
once_per_request = 100
API_KEY = os.getenv("API_KEY")
    # Define the CSV file name
payload = {}
headers = {
'Authorization': str(API_KEY)
}

df = pd.DataFrame([])
lastEvaluationKey = None
for i in range(max_page):
    if lastEvaluationKey:
        key_input = lastEvaluationKey
        pk = urllib.parse.quote(key_input['pk'])
        sk = urllib.parse.quote(key_input['sk'])
        url = "https://api.getgrass.io/devices?input=%7B%22limit%22:{},%22lastEvaluationKey%22:%7B%22pk%22:%22{}%22,%22sk%22:%22{}%22%7D%7D".format(once_per_request,pk,sk)
    else:
        url = "https://api.getgrass.io/devices?input=%7B%22limit%22:{}%7D".format(once_per_request)
    response = requests.request("GET", url, headers=headers, data=payload, verify=False)
    json_data = json.loads(response.text)
    # Extract the relevant data
    if 'result' in json_data:
        lastEvaluationKey = json_data['result']['data']['lastEvaluationKey']
        devices_data = json_data['result']['data']['data']
        # Create a DataFrame
        devices_data = pd.DataFrame(devices_data)
        df = pd.concat([df,devices_data], axis= 0)
        df = df.drop_duplicates()
    else:
        lastEvaluationKey = None


/home/aston/.venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.getgrass.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/aston/.venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.getgrass.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/aston/.venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.getgrass.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/aston/.venv/lib/python3.11/site-packages

In [8]:
df

,userAgent,version,ipAddress,ipScore,countryCode,deviceVersion,totalUptime,GSI1sk,entity,name,...,multiplier,GSI1pk,deviceId,modified,isConnected,totalPoints,connectionId,lastConnectedAt,userId,type
0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,51,34.92.250.88,0,HK,2.5.0,1312,CONNECTED#2hvfmFMSgc67U1s2ZFgs76WY4aG#f048dd5a...,UserDevice,Untitled Device,...,1,2h3JHQeoIEy1pwfdw3gzEDu0AE8,f048dd5a-67c4-3790-bdca-f89c116716f9,2024-06-15T19:27:08.871Z,True,0,qsUO9aDkiG4UIWp#h1PikYf0w-Q45lQpi2j1m,2024-06-15T19:27:07.000Z,2h3JHQeoIEy1pwfdw3gzEDu0AE8,extension
1,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,27,60.126.17.93,0,JP,2.5.0,231951,CONNECTED#2hvfvQXFwAMOqR22Udy2iQomjyt#97ca6eb6...,UserDevice,Untitled Device,...,1,2h3JHQeoIEy1pwfdw3gzEDu0AE8,97ca6eb6-0fe6-3ad2-a3af-19c47e8a4177,2024-06-15T19:28:21.076Z,True,0,uYpt3vTGDYO7ygy#3hGH9zQ8BwSifuaTeeapn,2024-06-15T19:28:19.000Z,2h3JHQeoIEy1pwfdw3gzEDu0AE8,extension
2,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,1,201.71.2.243,0,VE,2.5.0,0,CONNECTED#2hvgX6Vd2Dl2LvelDI7LhUP6JoU#ea78f120...,UserDevice,Untitled Device,...,1,2h3JHQeoIEy1pwfdw3gzEDu0AE8,ea78f120-7d9b-38b8-9422-1f84e6430df2,2024-06-15T19:33:21.231Z,True,0,niLb14RXNl1qnBO#vFDwoSreWb0LKYqYE1weV,2024-06-15T19:33:19.000Z,2h3JHQeoIEy1pwfdw3gzEDu0AE8,extension
3,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,9,142.54.231.38,0,US,2.5.0,0,CONNECTED#2hw298fDMOdWtBSpTZgUB5JFm5x#cab1b32d...,UserDevice,Untitled Device,...,1,2h3JHQeoIEy1pwfdw3gzEDu0AE8,cab1b32d-ff40-3fbc-b7cb-1c6b6608caf2,2024-06-15T22:31:04.499Z,True,0,BzDCwwDb38SBDpf#Hjmu_KKhHTuOJGOyq0Pm4,2024-06-15T22:31:03.000Z,2h3JHQeoIEy1pwfdw3gzEDu0AE8,extension
4,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,219,192.111.137.35,0,US,2.5.0,45153,CONNECTED#2hvz6iTNMr39AFj1q3RhjzWkvwY#99a6469f...,UserDevice,Untitled Device,...,1,2h3JHQeoIEy1pwfdw3gzEDu0AE8,99a6469f-238b-359f-9235-68700463e063,2024-06-15T22:06:05.391Z,True,0,qsUO9aDkiG4UIWp#GDI3Aui0g3XODfFF6HzkB,2024-06-15T22:06:03.000Z,2h3JHQeoIEy1pwfdw3gzEDu0AE8,extension
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,225,192.252.216.81,0,US,2.5.0,79561,CONNECTED#2hw5fqstdK7vIqGBsZ3PjQpYqvk#6e93005b...,UserDevice,Untitled Device,...,1,2h3JHQeoIEy1pwfdw3gzEDu0AE8,6e93005b-14bb-3a62-ac3e-336b0937b320,2024-06-15T23:00:05.011Z,True,0,bcu_xRjGXoqDNzW#yXhHfWwN9vp2BrbK0je0I,2024-06-15T23:00:03.000Z,2h3JHQeoIEy1pwfdw3gzEDu0AE8,extension
23,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,4040,189.240.60.168,0,MX,2.5.0,57,CONNECTED#2hw5oZo153ab9i8USM2GYwDkbpc#21c8b3e9...,UserDevice,Untitled Device,...,1,2h3JHQeoIEy1pwfdw3gzEDu0AE8,21c8b3e9-e863-3356-a0fa-a02179d605b6,2024-06-15T23:01:14.728Z,True,0,D7vlTmqxDW2xJKO#mJALCknhCSniyCjTVYsi5,2024-06-15T23:01:13.000Z,2h3JHQeoIEy1pwfdw3gzEDu0AE8,extension
54,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,2,91.185.236.239,0,RU,2.5.0,0,CONNECTED#2hw5nsyPkv4rvSQIQ2iEYw5kTvw#43322218...,UserDevice,Untitled Device,...,1,2h3JHQeoIEy1pwfdw3gzEDu0AE8,43322218-5e59-3233-92c1-5e36c57a80c0,2024-06-15T23:01:08.525Z,True,0,acb8mN_kGgQeym_#Efx-Dcs2b8dmPf3gCZydJ,2024-06-15T23:01:07.000Z,2h3JHQeoIEy1pwfdw3gzEDu0AE8,extension
56,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,3779,189.240.60.168,0,MX,2.5.0,57,CONNECTED#2hw5oJos74ES2pUQzyvlTht0H2U#82536e5e...,UserDevice,Untitled Device,...,1,2h3JHQeoIEy1pwfdw3gzEDu0AE8,82536e5e-4173-3eec-adac-cdb0f544efc4,2024-06-15T23:01:12.177Z,True,0,3WrNQKwNAbGA8zT#URYNXjUyCEgfvupkNiAMk,2024-06-15T23:01:10.000Z,2h3JHQeoIEy1pwfdw3gzEDu0AE8,extension


In [15]:
bad_ip_list = []
for indx, row in df.iterrows():
    ipAddress = row['ipAddress']
    ipScore = row['ipScore']
    totalUptime = row['totalUptime']
    if (totalUptime > 60) and (ipScore == 0):
        bad_ip_list.append(ipAddress)

In [16]:
# Write the bad IPs to bad_ip.txt
with open('bad_ip.txt', 'w') as bad_ip_file:
    for bad_ip in bad_ip_list:
        bad_ip_file.write(bad_ip + '\n')

# Write the bad IPs to bad_ip.txt
with open('bad_ip.txt', 'w') as bad_ip_file:
    for bad_ip in bad_ip_list:
        bad_ip_file.write(bad_ip + '\n')
        
# Read the IPs back from the file
with open('bad_ip.txt', 'r') as bad_ip_file:
    ips = bad_ip_file.readlines()

# Remove duplicates by converting to a set, then back to a list
unique_ips = list(set(ips))

# Sort the list to maintain a consistent order (optional)
unique_ips.sort()

# Write the unique IPs back to the file
with open('bad_ip.txt', 'w') as bad_ip_file:
    for ip in unique_ips:
        bad_ip_file.write(ip)

# Read the proxy list from good-grass-proxy-list.txt
proxy_list = read_proxy_list('good-grass-proxy-list.txt')

# Get the filtered list of proxy IPs
filtered_proxies = filter_proxies(proxy_list, bad_ip_list)
# Write the filtered proxies to proxy-list.txt
with open('good-grass-proxy-list-filterd.txt', 'w') as proxy_file:
    for proxy in filtered_proxies:
        proxy_file.write(proxy + '\n')